In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import base64
from requests import post,get
import json
import numpy as np
import pandas as pd
import utils
utils.check_credentials()

In [17]:
lista_paises={'Los 40 Principales':"6hN982OqftxyShJ28FcAIc",'UK Top 100':"2JyhlK34xSSXR4OQ3WflEq",'Billboard Hot 100':"6UeSakyzhiEt4NB3UAd6NQ",'Shazam Top 200':'1YvHZR6lwysBB6UNUa800r'}

In [18]:
token = utils.get_token()
print("¡Token obtenido exitosamente!")
print(f"Token (primeros 20 caracteres): {token[:20]}...")

¡Token obtenido exitosamente!
Token (primeros 20 caracteres): BQACL-BzS4okhylAPIjT...


In [23]:
columns = ["top", "track_name", "track_popularity", "duration_ms", "artist_name", 
           "artist_genres", "artist_popularity", "feats", "explicit", "album", 
           "type", "relase_date", "track_id", "artist_id", "list"]
df_playlist = pd.DataFrame(columns=columns)

In [24]:
jsons = []
for playlist_name, playlist_id in lista_paises.items():
    playlist_json = utils.get_playlist(token, playlist_id)
    playlist_json['playlist name']=playlist_name
    jsons.append(playlist_json)

In [25]:
for json in jsons:
    current_playlist_name=json['playlist name']
    track_info = json["tracks"]["items"]
    playlist_data = []
    for i in range(len(json["tracks"]["items"])):
        feats = []
        data = {
            "top": i+1,
            "track_name": track_info[i]["track"]["name"],
            "track_popularity": track_info[i]["track"]["popularity"],
            "artist_name": track_info[i]["track"]["artists"][0]["name"],
            "artist_genres": "",
            "duration_ms": track_info[i]["track"]["duration_ms"],
            "artist_popularity": "",
            "feats": "",
            "explicit": track_info[i]["track"]["explicit"],
            "album": track_info[i]["track"]["album"]["name"],
            "type": track_info[i]["track"]["album"]["album_type"],
            "relase_date": track_info[i]["track"]["album"]["release_date"],
            "track_id": track_info[i]["track"]["id"],
            "artist_id": track_info[i]["track"]["artists"][0]["id"],
            "list": current_playlist_name
            }

        artist_info = utils.get_artist(token, data["artist_id"])
        data["artist_popularity"] = artist_info["popularity"]

        if len(artist_info["genres"]) == 1:
            data["artist_genres"] = artist_info["genres"][0]
        elif len(artist_info["genres"]) > 1:
            generes = artist_info["genres"]
            data["artist_genres"] = "|".join(generes)
        else:
            data["artist_genres"] = np.nan

        if len(track_info[i]["track"]["artists"]) > 1:
            for j in range(len(track_info[i]["track"]["artists"])-1):
                feats.append(track_info[i]["track"]["artists"][j+1]["name"])
                data["feats"] = "|".join(feats)
        else:
            data["feats"] = np.nan
        feats = []

        playlist_data.append(data)

        
    # CONCAT
    df_temp = pd.DataFrame(playlist_data)
        
    df_playlist = pd.concat([df_playlist, df_temp], ignore_index=True)

In [26]:
df_playlist.to_csv("top_charts.csv", index=False, encoding='utf-8')
print("CSV exportado")

CSV exportado
